Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
COLLABORATORS = ""

---

# CIS 545 Homework 5 and 6: Amazon Review Analysis and Classification

Your main training set for this assignment is the text from 100,000 reviews from Amazon.com, their timestamps, and their star ratings. The high level goal of this homework is to use the textual and temporal data to predict the star ratings.

**Adventurers beware!** Analyzing this data in `sklearn` will likely kill your kernel. So instead we will use the package [gensim](https://radimrehurek.com/gensim/) for analysis. gensim specializes in efficient implementations of common modeling techniques for big text.

In [ ]:
# install stuff
!pip install gensim
!pip install paramiko

In [ ]:
# import stuff
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim import corpora
from gensim.models import LsiModel, KeyedVectors
import sklearn.model_selection as ms
from sklearn.ensemble import RandomForestClassifier
from datetime import *

In [ ]:
# read stuff
reviews_dict = corpora.Dictionary.load("reviews.dict")
reviews_bow = corpora.MmCorpus('train_reviews.mm')
reviews_times  = np.load('train_times.npy')
reviews_times.shape = (len(reviews_bow),1)
reviews_wc = corpora.MmCorpus('reviews_wc.mm')
reviews_sppmi_300 = np.load('reviews_sppmi_300.npy')
reviews_wv = KeyedVectors.load("word_vectors.wv", mmap='r')
y = np.vstack((np.repeat(1, 4000), np.repeat(2, 4000), np.repeat(3, 4000), np.repeat(4, 4000), np.repeat(5, 4000)))
y = np.repeat(y, 5)

## Step 0: Format Exploring

We will start with exploring the format of all of the data files that we imported above. 

### Step 0.1: gensim dictionary (lexicon)

Most data science over text has some form of vocabulary. Simply put, you need to decide which words your model will care about. Very rare words, misspellings, numbers, and urls are good candidates for exclusion, especially since if the model needs any form of normalization, the time complexity of such computations is at least linear in the size of the vocabulary, if not worse.

A lexicon associates each word in the vocabulary with an index. Since words are repeated, the model can save space by using the index for every repetition and only linking the index with the string form once. A `gensim` dictionary is special in that it is very fast and allows bidirectional lookups, namely, word to index and index to word.

After reviewing the [documentation](https://radimrehurek.com/gensim/corpora/dictionary.html), rewrite the right hand side of each line in the cell below with the answers to these questions.

1. What is the index of "good"? Look it up and store it in a variable named `good`. To clarify, if you find that 42 is the index of "good", change the line below so that it sets `good` equal to 42. Of course, you can do this with `good = 42` and earn full points, but it is a litte better to reuse the command with which you found the index. For example, if the `gensim` dictionary worked like a list of strings, you could do it with  
`good = reviews_dict.iloc('good')`.
2. What word belongs to index 195? Look it up and store it in a variable named `oneninefive`.
3. What happens when you evaluate `reviews_dict[i]` for some variable `i`? If this returns the word associated with that index, set `idx2word` to `True`. Otherwise, set it to `False`. For example, if `reviews_dict['good']` equals `good`, `idx2word` should be `False`, but if `reviews_dict[195]` equals `oneninefive`, `idx2word` should be `True`.

Hint: `token2id('good')` and `id2token(195)` didn't work for me either. Keep trying!

In [ ]:
reviews_dict.token2id['good']

In [ ]:
# dictionary_answer
good = 19
oneninefive = "brass"
idx2word = True
reviews_dict[195]

In [ ]:
# dictionary_test_1 (5 points)
print("The index of good is", good)

In [ ]:
# dictionary_test_2 (3 points)
print("Word 195 is", oneninefive)

In [ ]:
# dictionary_test_3 (2 points)
print("It is", idx2word, "that brackets can look up a word by its index.")

### Step 0.2: Efficiently storing text

`gensim` represents everything in a **sparse** way. Namely, the representation of a review will be a variable-size list that contains counts of the words that _are present_ in the review. A **dense** representation, on the other hand, such as a matrix, would, in addition to the present words, contain zero counts for all of the words that are not in that particular review. For some examples, see [this tutorial](https://radimrehurek.com/gensim/tut1.html).

You may run the cell below as it is to see what is actually stored in a gensim [corpus](https://radimrehurek.com/gensim/corpora/mmcorpus.html). In each review, `gensim` stores a tuple of size 2 for each distinct word in the review. The first number in the tuple is the index of the word in the dictionary and the second number in the tuple is the count of the times that word appeared in that review.

Now, we would like you to add a bit of code to the cell below that "translates" the first two amazon reviews into something more readable. The reviews are already represented as bags of words, so recall that you cannot recover the order of the words in the reviews. Rather, the words are in alphabetical order. But, we would like you to spell out the repeats of each word. So, if the original review were "to be or not to be", `reviews_bow` would have something like:

`[(0, 2.0), (1, 1.0), (2, 1.0), (3, 2.0)]`

and we would like you to make the string

`"be be not or to to"`

When you are done, `readable_reviews[0]` should be the first review and `readable_reviews[1]` should be the second.

Hint: In a normal array, you would be able to access the first two reviews with the indices 0 and 1, but this is not a normal array. Therefore, we have started an iteration through the documents and put a break the loop when `i` reaches a value strictly greater than 1. 

In [ ]:
# readable_answer
i = 0
readable_reviews = []
for doc in reviews_bow:
    review = ''
    if i > 1: break
    print(doc)
    i += 1
    for (s, j) in doc:
      for k in range(int(j)):
        review = review + reviews_dict[s]+' '
    readable_reviews.append(review)

In [ ]:
reviews_bow

In [ ]:
reviews_dict[1]

In [ ]:
# readable_test_1 (5 points)
print(readable_reviews[0])

In [ ]:
# readable_test_2 (5 points)
print(readable_reviews[1])

### Step 0.3: Parsing review times

It might be useful in predicting the scores of the reviews to know when the reviews were written. In this dataset, the day of the review was recorded as the number of seconds that passed between midnight on January 1, 1970 (the beginning of time for many computer systems) and the time the review was created. This may be efficient because it is one integer, but it is not very convenient. So we are going to convert these int objects to [datetime](https://docs.python.org/3/library/datetime.html) objects:

1. Do not change `review_times` in any way. Work with other variables instead.
1. Make a new variable named `converted_times`.
2. Set that variable equal to a pandas `Series` object made from `review_times` but the entries should be of type `datetime` or `Timestamp`.

3. Make a new variable named `forty_days_before_review_times_0`. 
4. Set that variable equal to 40 days before the time of the first review, using the `timedelta` function.

Hint: You might find `datetime.fromtimestamp` to be useful.

In [ ]:
# datetime_answer 
# YOUR CODE HERE
reviews_times_c = reviews_times.copy()
converted_times = []

In [ ]:
for i in reviews_times_c:
  converted_times.append(datetime.fromtimestamp(i[0]))
converted_times = pd.Series(converted_times)

In [ ]:
# datetime_test_1 (2 points)
print("converted_times is a", type(converted_times))

In [ ]:
# datetime_test_2 (5 points)
print("converted_times[0] is a", type(converted_times[0]))

In [ ]:
from datetime import timedelta
forty_days_before_review_times_0 = converted_times[0]+timedelta(days=-40)

In [ ]:
# datetime_test_3 (3 points)
display(converted_times[0])
display(forty_days_before_review_times_0)

## Step 1: Word vectors

We saw in class that performing several optimizations on a word-context shifted positive pointwise mutual information ($SPPMI$) matrix, including decomposing with SVD, is mathematically equivalent to a neural network that learns word embeddings using skip gram with negative sampling ($SGNS$).  We are going to implement each of these here and compare against a pure bag of words ($BOW$) model as a baseline.

To begin, let's make a corpus out of our favorite toy dataset: the 5 computer science and 4 math article titles. After lower casing, tokenizing, and stop wording, the corpus looks like `titles` in the cell below. Then, we create a dictionary and a sparse document-term matrix.

In [ ]:
titles = [['human', 'interface', 'computer'],
          ['survey', 'user', 'computer', 'system', 'response', 'time'],
          ['eps', 'user', 'interface', 'system'],
          ['system', 'human', 'system', 'eps'],
          ['user', 'response', 'time'],
          ['trees'],
          ['graph', 'trees'],
          ['graph', 'minors', 'trees'],
          ['graph', 'minors', 'survey']]

titles_dict = corpora.Dictionary(titles)
titles_bow = [titles_dict.doc2bow(title) for title in titles]
display(titles_bow)

### Step 1.1: Sparse to dense

To get the term-document matrix that we have seen in lecture, we need to convert this matrix to its dense form. Write a function `densify` that takes as input:

1. a sparse matrix in the format of `titles_bow` above
3. an integer number of columns

and returns a NumPy array. Note that `titles_bow` is a document-term matrix, not a term-document matrix, so we transpose it in the test cell to show the matrix from lecture (with the rows and columns slightly reordered).

In [ ]:
# densify_answer
def densify(sparse, columns):
  num_row = len(sparse)
  td = np.zeros((num_row, columns))
  k=0
  for item in sparse:
    for (i,j) in item:
      td[k, i] = j
    k=k+1
  return td
      
  

In [ ]:
# densify_test_1 (5 points)
td = densify(titles_bow, len(titles_dict)).transpose()
print(td)

In [ ]:
# densify_test_2 (5 points)
print(td.shape)

### Step 1.2: Counting words

Write a function `count_words` that takes as input:

1. a dictionary in the format of `titles_dict` above
2. a bag of words corpus in the format of `titles_bow` above

and returns a list called `counts` that has the total occurrences of each word in the corpus, in the order of the original word indices. Note that a gensim dictionary only counts a word once per document, so we can't directly use `titles_dict`.

In [ ]:
# count_words_answer
def count_words(gsdict, gsbow):
  num_row = len(gsbow)
  num_col = len(gsdict)
  td = np.zeros((num_col)).astype(int)
  for doc in gsbow:
    for (i,j) in doc:
      td[i] +=j
  return td

In [ ]:
# count_words_test_1 (5 points)
titles_counts = count_words(titles_dict, titles_bow)
display(len(titles_counts))

In [ ]:
# count_words_test_2 (5 points)
display(titles_counts)

The cell below makes the counts for the 100,000 Amazon reviews.

In [ ]:
reviews_counts = count_words(reviews_dict, reviews_bow)

### Step 1.3: The word-context matrix

Now, write a function `word_context` that takes as input:

1. a dictionary in the format of `titles_dict` above
2. a corpus in the format of `titles` above
3. a window size (integer)

and creates a **sparse** word-context matrix. Recall that the word-context matrix is a square matrix and the number of rows/columns is the number of words in the vocabulary/dictionary. A nonzero term in this matrix represents the number of times word $i$ appears within (and including) the window size distance of word $j$. The word-context matrix for this toy corpus is in the lecture slides, so you may use that to check your work.

_Note: the only time a word should be in the context of itself is when it occurs with a second instance of that word, for example in the fourth "document" of the titles corpus._

In [ ]:
# word_context_answer
def word_context(gsdict, gscorpus, window):
  size = len(gsdict)
  wc = np.zeros((size,size)).astype(int)
  count = wc.copy()
  new_wc = wc.copy()
  for corpus_list in gscorpus:
    for idx in range(len(corpus_list)):
      low, high = idx -window, idx+window
      if (idx-window)< 0:
        low = 0
      if (idx+window+1> len(corpus_list)):
        high = len(corpus_list) -1
      for idx2 in range(low, high+1):
        i = gsdict.token2id[corpus_list[idx2]]
        j = gsdict.token2id[corpus_list[idx]]
        if (idx2 != idx) and (count[i,j]==0) :
          wc[i, j] +=1
#         elif (idx2 != idx) & (count[i,j]!=0):
#           new_wc[i,j] +=1
#       for i in range(size):
#         for j in range(size):
#           if wc[i,j]!=0:
#             count[i,j] = 1
#           if new_wc[i,j]>wc[i,j]:
#             wc[i,j] = new_wc[i,j]
#       new_wc =  np.zeros((size,size)).astype(int)
      #print(wc[5,2]), print(new_wc[5,5])
  wc_sparse = []
  for i in range(size):
    row_sparse = []
    for j in range(size):
      if wc[i,j]!=0:
        row_sparse.append((j, wc[i,j]))
    wc_sparse.append(row_sparse)
  return wc_sparse
      

In [ ]:
# word_context_test_1 (5 points)
titles_wc = word_context(titles_dict, titles, 2)
display(titles_wc)

In [ ]:
titles_dict[0]

In [ ]:
titles = [['human', 'interface', 'computer'],
          ['survey', 'user', 'computer', 'system', 'response', 'time'],
          ['eps', 'user', 'interface', 'system'],
          ['system', 'human', 'system', 'eps'],
          ['user', 'response', 'time'],
          ['trees'],
          ['graph', 'trees'],
          ['graph', 'minors', 'trees'],
          ['graph', 'minors', 'survey']]

In [ ]:
# word_context_test_2 (5 points)
display(densify(titles_wc, len(titles_dict))) 

### Step 1.4: Enhancing the word-context matrix

In lecture, we saw a number of possible enhancements for word-context matrices. Write a function `sppmi` that takes as input:

1. a sparse word-context matrix in the format of `titles_wc` above
3. a counts dictionary
4. a float `logk`

and returns a new sparse word-context matrix with the values in the matrix replaced by shifted positive pointwise mutual informations ($SPPMI$). The formula is:

$$SPPMI = \max(\log (\frac{\#(w,c) |D|}{\#(w)\#(c)}) - \log(k), 0)$$

where $\#(w,c)$ is the count of word $w$ in context $c$ (the original count from the last section), $\#(w)$ is the count of word $w$, $\#(c)$ is the count of word $c$ (both come from `titles_counts`), $|D|$ is the length of the corpus, and $k$ is a free hyperparameter.

In [ ]:
# sppmi_answer
def sppmi(gswc, counts, logk):
  D = sum(counts)
  size = len(counts)
  sppmi = np.zeros((size, size))
  row = 0
  sparse_mi = []
  for doc in gswc:
    row_mi = []
    for (i,j) in doc:
      sppmi = max(np.log(j*D/counts[i]/counts[row]) - logk, 0)
      if sppmi !=0:
        row_mi.append((i,sppmi))
    row = row + 1
    sparse_mi.append(row_mi)
  return sparse_mi
  

In [ ]:
# sppmi_test_1 (5 points)
titles_sppmi = sppmi(titles_wc, titles_counts, 0)

In [ ]:
# sppmi_test_1 (5 points)
titles_dense_sppmi = densify(titles_sppmi, len(titles_dict))
display(titles_dense_sppmi.round(1))

The cell below converts the counts to $SPPMI$s in the word-context matrix for the 100,000 Amazon reviews.

In [ ]:
reviews_sppmi = sppmi(reviews_wc, reviews_counts, np.log(5))

### Step 1.5: Sparse SVD/PCA/LSA/LSI

One of the greatest benefits of gensim is that there is no need to densify the matrix before decomposing. Indeed, they post some impressive numbers about their SVD speed [here](https://radimrehurek.com/gensim/models/lsimodel.html). But for now, we will test it just on our toy dataset. In the cell below, write a function called `reconstruction` that takes as input:

1. a sparse matrix
2. a gensim dictionary
2. a cutoff for PCA

The function should compute the norm of the difference of the reconstructed matrix and the original, and return that dividing by the norm of the original.

Hint: The right singular vectors ($V$ or `model[sparse]`) already contain the singular values ($S$ or `model.projection.s`) so don't include them again!

In [ ]:
# reconstruction_answer
def reconstruction(sparse, gsdict, cutoff):
  model = LsiModel(sparse, id2word = gsdict, num_topics = cutoff)
  V = model[sparse]
  U = model.projection.u
  dense_V = densify(V, cutoff).transpose()
  dense = densify(sparse, len(gsdict)).transpose()
  #print(dense.shape)
  #recon = U[:,0:cutoff-1].dot(dense_V[0:cutoff-1, :])
  recon = U.dot(dense_V)
  norm_re = np.linalg.norm(recon - dense)
  norm_or = np.linalg.norm(dense)
  return norm_re/norm_or
  

cutoff = 2

In [ ]:
# reconstruction_test_1 (5 points)

error = reconstruction(titles_bow, titles_dict, cutoff)
print("The reconstruction error with", cutoff, "components on the the toy dataset is", error)

In [ ]:
# reconstruction_test_2 (5 points)
cutoff = 9
error = reconstruction(titles_bow, titles_dict, cutoff)
print("The reconstruction error with", cutoff, "components on the the toy dataset is", error)

### Step 1.6: Assembling the dense representation

The last step in this process is to combine our two datasets, namely the document-term matrix and the word-context matrix. The appeal is that rather than reducing dimensionality by choosing a very small vocabulary, we can select relevant features from a vector space that contains all of the words. However, this requires a heavy assumption: a document representation is the sum of the representations of its words. This does not allow any non-compositionality of language (which we know there is some) and it makes each word equally important (which arguably is false as well).

In the cell below, write a function `vec2doc` that takes as input:

1. a bag of words corpus in the format of `titles_bow` above
2. a list of word vectors in proper order (by dictionary index) like `reviews_wv` above

and returns a dense matrix. This matrix contains one vector per document which was computed by summing all of the vectors corresponding to the words in the document (including repeats).

In [ ]:
# vec2doc_answer
def vec2doc(gsbow, vectors):
  return_vecs = []
  for doc in gsbow:
    sum = np.zeros(300)
    for (i,j) in doc:
      sum = sum + j * vectors[i]
    return_vecs.append(sum)
  return np.array(return_vecs)
      

In [ ]:
# vec2doc_test_1 (5 points)
vecs_sppmi = vec2doc(reviews_bow, reviews_sppmi_300)

In [ ]:
# vec2doc_test_2 (5 points)
display(vecs_sppmi.shape)

## Step 2: Feature selection and hyperparameter tuning

By this point, you have already written most of the code to import and use the word embeddings made by `word2vec`. Once both are loaded and the hyperparameters are set, you are ready to compare them against the bag of words baseline on the classification task.

### Step 2.1: Assembling the other dense representation

The only small structural difference between the word vectors that you computed and the `word2vec` embeddings is that your word vectors use a vocabulary/dictionary for indexing, whereas `word2vec` vectors are accessed using a Python dictionary. So, to access the vector for the word "good" you would need to use the integer `good` in `vecs_sppmi[good]` and the string 'good' in `reviews_wv['good']`.

Since string lookups can be inefficient and so you may maximally reuse your code, write a function `use_dict` that takes as input:

1. a dictionary in the format of `titles_dict` above
2. `KeyedVector word2vec` embeddings, as in `reviews_wv`

and returns a list of vectors indexed using the input dictionary.

In the unlikely case that you are missing a vector, just set the value at that index to the integer `0`.

In [ ]:
len(reviews_dict)

In [ ]:
# use_dict_answer
def use_dict(gsdict, vectors):
  vecs = []
  for i in range(len(gsdict)):
    word = gsdict[i]
    if word not in vectors.vocab:
      vecs.append(np.zeros(300))
    else:
      vecs.append(vectors[word])
  return np.array(vecs)

In [ ]:
# use_dict_test (5 points)
reviews_sgns_300 = use_dict(reviews_dict, reviews_wv)
vecs_sgns = vec2doc(reviews_bow, reviews_sgns_300)

### Step 2.2: PCA on pure BOW

While the $SPPMI$ and $SGNS$ models are already dimensionality reduced, it would be a good idea to also run PCA on the $BOW$ baseline. Train a gensim `LsiModel` on `reviews_bow` using `reviews_dict` as the dictionary and 1000 components.

In [ ]:
# pca_bow_answer
max_cutoff = 1000
model =  LsiModel(reviews_bow, id2word = reviews_dict, num_topics = max_cutoff)

Once it is finished, use the code below to plot the explained variance versus number of components. You just need to pass in a list of singular values (no `diag` necessary).

In [ ]:
def plot_variance_vs_features(singular_values, cutoff):
    evr = np.array([singular_values[i]**2 / sum(singular_values**2) for i in range(cutoff)])
    var = np.cumsum(evr*100)
    plt.ylabel('% Variance Explained')
    plt.xlabel('# of Features')
    plt.title('PCA Analysis')
    plt.style.context('seaborn-whitegrid')
    plt.plot(var)
    
plot_variance_vs_features(model.projection.s, max_cutoff)

The good news is this curve is very steep in the beginning, which shows that a lot of information is conveyed in the first components. However, there is no plateau that we can use to choose a cutoff!

**So, instead, we will break off a validaton set and use classifier performance to tune this hyperparameter.**

In [ ]:
def evaluate_model(X, review_times, y):
    X = np.hstack((X, review_times))
    X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.2, random_state = 195)
    rfor = RandomForestClassifier(n_estimators=51, random_state=195)
    rfor.fit(X_train, y_train)
    return rfor.score(X_test, y_test)

### Step 2.3: Choosing the number of components via the downstream task

In some ways, this method of choosing the number of components is even better than the plateau method, because we are optimizing directly on the machine learning task rather than something intrinsic to the dataset. In the cell below, call `evaluate model` on models between 10 and 30 PCA components. For each model, you will need to train an `LsiModel`, compute the $V$ matrix (right singular vectors), call `densify` on that, and pass the dense matrix to evaluate model. Store all of your accuracies in a list named results.

In [ ]:
# # baseline_tuning_answer
accuracy = []
for i in range(10,31):
  model =  LsiModel(reviews_bow, id2word = reviews_dict, num_topics = i)
  V = model[reviews_bow]
  dense_V = densify(V, i)
  acc = evaluate_model(dense_V, reviews_times, y)
  accuracy.append(acc)



In [ ]:
#accuracy

In [ ]:
accuracy

In [ ]:
# dense_V.shape

In [ ]:
# U = model.projection.u
# U.shape

In [ ]:
# baseline_tuning_test (5 points)
# display(results)

In [ ]:
# final_sppmi_test (5 points)
final_sppmi = evaluate_model(vecs_sppmi, reviews_times, y)
print(final_sppmi)

In [ ]:
# final_sgns_test (5 points)
final_sgns = evaluate_model(vecs_sgns, reviews_times, y)
print(final_sgns)

## Step 3: In-class Kaggle competition

To conclude this homework and this class, we are opening up to a more flexible system. We would like for you to use everything you have learned in CIS 545 to make the best star rating predictor possible. For grading purposes, we are going to check (manually) whether you thought about and worked on each of the subsections that follow. These sections will be pass/fail and the questions under each of the headlines are meant to help you brainstorm, not impose requirements.

**Beyond that, a small amount of extra credit will be awarded to the students with the best performing models. We should be able to maintain an (anonymous) leaderboard on Piazza. Scores on this leaderboard will correspond to performance on our test set, which has not been released to you. Therefore, our TAs will run your models on our test data and report the results to ensure fairness.**

### Step 3.1: Scaling

What is the range of values for each feature? Are they comparable? Do some dominate others? Try minmax, standard, log, exponential, binary, or thresholding. How are predicted 1 star rating and predicted 5 star ratings different? How are actual 1 star ratings and actual 5 star ratings different? 

In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [ ]:
#range of feature value of second feature
max(vecs_sgns[:,1]) - min(vecs_sgns[:,1])

In [ ]:
#did a few trials on difference in feature values across ratings
np.mean(vecs_sgns[0+20000:2000+20000, 0] - vecs_sgns[2000:4000, 0]) # difference of first feature in 1 star rating(same star rating)

In [ ]:
np.mean(vecs_sgns[0:4000, 0] - vecs_sgns[4000:8000, 0]) # difference of first feature between 1 star and 2 star rating

In [ ]:
np.mean(vecs_sgns[0:4000, 0] - vecs_sgns[16000:20000, 0]) # difference of first feature between 1 star and 5 star rating

In [ ]:
#I also did trials on other features. 
#The results show that feature difference within the same rating would be smaller than feature difference between different star ratings.

In [ ]:
#try minmax scaler
def minmax(col):
  scaler = MinMaxScaler()
  new_col = scaler.fit_transform(col)
  return new_col.reshape(-1)
#try standard scaler
def standard(col):
  scaler = StandardScaler()
  new_col = scaler.fit_transform(col)
  return new_col.reshape(-1)
#save copies of scaled variables
vecs_sgns_c = vecs_sgns.copy()
vecs_sppmi_c = vecs_sppmi.copy()
vecs_sgns_c2 = vecs_sgns.copy()
vecs_sppmi_c2 = vecs_sppmi.copy()
for i in range(vecs_sppmi_c.shape[1]):
  vecs_sgns_c[:,i] = minmax(vecs_sgns[:,i].reshape(-1,1))  #apply scaler to each feature
  vecs_sppmi_c[:,i] = minmax(vecs_sppmi[:,i].reshape(-1,1)) #apply scaler to each feature
  vecs_sgns_c2[:,i] = standard(vecs_sgns[:,i].reshape(-1,1)) #apply scaler to each feature
  vecs_sppmi_c2[:,i] = standard(vecs_sppmi[:,i].reshape(-1,1)) #apply scaler to each feature

  

In [ ]:
#minmax scaler results
final_sgns_scale = evaluate_model(vecs_sgns_c, reviews_times, y)
final_sppmi_scale = evaluate_model(vecs_sppmi_c, reviews_times, y)


In [ ]:
#standard scaler results
final_sgns_scale2 = evaluate_model(vecs_sgns_c2, reviews_times, y)
final_sppmi_scale2 = evaluate_model(vecs_sppmi_c2, reviews_times, y)


In [ ]:
print(final_sgns_scale)
print(final_sppmi_scale)
#results using minmax scaler show that sppmi matrix works better on random forest model

In [ ]:
print(final_sgns_scale2)
print(final_sppmi_scale2)
#results using standard scaler also show that sppmi matrix works better on random forest model

### Step 3.2: Clustering

What is the distribution of ratings? How else might this dataset be artificially balanced? What patterns can you find? Are there detectable clusters? Are some ratings easier to classify than others?

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
#kmeans, set n_clusters =5, corresponding to star 1 - 5 rating
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5)
pca_sppmi = PCA(n_components=2).fit_transform(vecs_sppmi)
pca_sgns = PCA(n_components=2).fit_transform(vecs_sgns)
kmeans.fit(pca_sppmi)
y_kmeans_sppmi = kmeans.predict(pca_sppmi)
plt.scatter(pca_sppmi[:, 0], pca_sppmi[:, 1], c=y_kmeans_sppmi, s=50, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);
##################
plt.figure()
kmeans.fit(pca_sgns)
y_kmeans_sgns = kmeans.predict(pca_sgns)
plt.scatter(pca_sgns[:, 0], pca_sgns[:, 1], c=y_kmeans_sgns, s=50, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);


In [ ]:
#By doing PCA on vecs_sppmi and vec_sgns and pick top 2 PCs, we can see that there are clear clusters.
#However, it doesn't mean that in each cluster, the ratings are the same. So I did some checks on the majority 
#cluster of each rating. Cluster numbers are in range(5)

In [ ]:
np.argmax([list(y_kmeans_sppmi[0:4000]).count(i) for i in range(5)]) # majority for rate 1 is in this cluster

In [ ]:
np.argmax([list(y_kmeans_sppmi[0+4000:4000+4000]).count(i) for i in range(5)])# majority for rate 2 is also in this cluster

In [ ]:
np.argmax([list(y_kmeans_sppmi[0+16000:4000+16000]).count(i) for i in range(5)]) #so does rate 5, so there is no detectable clusters when using vecs_sppmi or vecs_sgns directly sa features

In [ ]:
# By doing kmeans only using embedding features, I observed that reviews of all star ratings will tend to 
# cluster at the same cluster. So there is no clear detectable clusters.

In [ ]:
def evaluate_kmeans_model(X, review_times, y):
    X = np.hstack((X, review_times))
    kmeans = KMeans(n_clusters=5)
    X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.2, random_state = 195)
    kmeans.fit(X_train, y_train)
    return kmeans.score(X_test, y_test)

In [ ]:
#By purely conducted k-means, the accuracy score is really bad.
kmeans_score = evaluate_kmeans_model(pca_sppmi, reviews_times, y)

In [ ]:
kmeans_score

### Step 3.3: Regression

In the baseline models, star ratings were treated as categorical variables. So the models had no sense that a rating of 4 stars is in between ratings of 3 stars and 5 stars. What are the advantages and disadvantages of that approach? If you reformulate the problem as regression, how do you measure success? Is that way of measuring fair to the classifiers?

In [ ]:
# linear regression model with regularization
from sklearn.linear_model import LinearRegression
from sklearn import linear_model


In [ ]:
def evaluate_reg_model(X, review_times, y):
    X = np.hstack((X, review_times))
    X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.2, random_state = 195)
    reg = linear_model.Ridge(alpha=0.1).fit(X_train, y_train)  #adding regularization term to reduce over-fitting
    return reg.score(X_test, y_test)

In [ ]:
reg_score = evaluate_reg_model(vecs_sppmi, reviews_times, y)

In [ ]:
reg_score # accuracy score using vecs_sppmi is 0.3638

In [ ]:
reg_score = evaluate_reg_model(vecs_sgns, reviews_times, y)
print(reg_score) # accuracy score using vecs_sgns is 0.3476

In [ ]:
# linear regression with penality still doesn't have a good performance on categorical variable classification.
# The advantage of using regression is to break the hard wall among different ratings. 
# But since the predicted labels are still categorical and deterministic, using regression here actually diminish the actual difference between each ratings.
# If I can reformulae the problem as regression, I will label y as a 5-column matrix with each column as the probability in each star rating. 
# Linear regression would perform much better in continuous data, and log loss would be a good metric to judge the performance.

### Step 3.4: Time Series

Since real products were rated over time, these star ratings were likely subject to trends. Can these trends be leveraged? What curve would explain the trends without requiring too many parameters? Can you use an autoregressor? Can you use a moving average? Are the data stationary?

In [ ]:
import statsmodels
from statsmodels.tsa.stattools  import adfuller

In [ ]:
#formulate Series data
times = pd.DataFrame(converted_times)
rating = pd.DataFrame(y)
data = pd.concat([times, rating], axis = 1)
data.columns = ['times', 'rating']
data = data.set_index('times').sort_index()
data_series= pd.Series(data.iloc[:, 0])

In [ ]:
data

In [ ]:
# calculate rolling mean and standard deviation
rolmean = data_series.rolling(window = 12).mean()
rolstd = data_series.rolling(window = 12).std()

In [ ]:
# rolstd = pd.Series(np.nan_to_num(rolstd))

In [ ]:
#plot original data with rolling mean and standard deviation
plt.plot(data_series, label ='original')
plt.plot(rolmean, color='red', label ='rolling means')
plt.plot(rolstd, color = 'black', label = 'rolling std')
plt.legend(loc='best')

In [ ]:
data_log = np.log(data_series) # take log of the original data

In [ ]:
#using dickey-fuller test to examine whether the data is stationary or not. 
#the resulting p value is zero, which means that the data is stationary.
adfuller(data_series)


In [ ]:
#try the test on log data, which also give zero p value, meaning the log data is stationary
adfuller(data_log)

In [ ]:
# def evaluate_model(review_times, y):
#     #X = np.hstack((X, review_times))
    
#     #X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.2, random_state = 195)
#     #rfor = RandomForestClassifier(n_estimators=51, random_state=195)
#     #rfor.fit(X_train, y_train)
#     return rfor.score(X_test, y_test)

### Step 3.5: Neural networks

The baseline models were are random forests. What other algorithms could be used? In a neural network, how many layers and nodes? Any convolutions or recurrences? How do you keep the model from overfitting? What regularization is appropriate? Is the task too hard or too easy for a neural network?

In [ ]:
# To conclude on neural network, it seems that neural network is not very suitable for this task. Since the only
# features we used here are embedding features and time feature, adding these to neural network may over-complicate 
# the problem. 

In [ ]:
# use sklearn to train a multi-layer percetron as neural-net baseline
from sklearn.neural_network import MLPClassifier
def nn_model(X, review_times, y):
    X = np.hstack((X, review_times))
    X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.2, random_state = 195)
    nn = MLPClassifier(hidden_layer_sizes=(100, ), activation='relu', solver='adam', 
                       alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.01, 
                       power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, 
                       verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, 
                       early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, 
                       epsilon=1e-08, n_iter_no_change=10) # using the default multi-layer perceptron model
    nn.fit(X_train, y_train)
    print('\nTRAIN SCORE', nn.score(X_train, y_train))
    print('TEST SCORE', nn.score(X_test, y_test))
    return None

nn_model(vecs_sppmi, reviews_times, y)

In [ ]:
!pip install keras
!pip install tensorflow


In [ ]:
#using keras to build my own neural-network, using minmax scaled data
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, Embedding,GlobalMaxPooling1D
#create model
def nn_model(X, review_times, y):
    #X = np.hstack((X, review_times)) #it seems review_times doesn't help in this neural network
    X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.2, random_state = 195)
    embedding_dim = 50
    vocab_size = len(X_train)
    maxlen = 300
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=maxlen)) #embedding layer
    model.add(Conv1D(64, 5, activation='relu')) #convolution layer
    model.add(GlobalMaxPooling1D()) # pooling
    model.add(Dense(10, activation='relu')) #fully connected layer
    model.add(Dense(1, activation='softmax')) #fully connected layer
    model.compile(optimizer='adam', 
                  loss='binary_crossentropy',
                  metrics=['accuracy']) 
    model.summary()
    history = model.fit(X_train, y_train,
                    epochs=3,
                    validation_data=(X_test, y_test),
                    batch_size=100)
    loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
    print("Training Accuracy: {:.4f}".format(accuracy))
    loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
    print("Testing Accuracy:  {:.4f}".format(accuracy))
    return None
  
nn_model(vecs_sppmi_c, reviews_times, y)

### Step 3.6: Other enhancements

You may use any other techniques from this class and beyond. Feel free to explore!

In [ ]:
#In this section, I'm basically exploring more methods such as logistic regression and boosted tree. I'm assuming essemble method would have a better performace on embedding 
#features than random forest does. And boosted tree seems to work well when the feature dimension is relatively large. I'm also trying mixed model, meaning to combine the results from
#logistic regression and boosted tree together and see whether the mixed results would have a higher accuracy. 

In [ ]:
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
def evaluate_log_model(X, review_times, y):
    #X = np.hstack((X, review_times)) # reviews_times doesn't help the model.
    X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.2, random_state = 195)

    model = LogisticRegression(C=1e-2, random_state=17, solver='lbfgs', 
                           max_iter=300,
                          n_jobs=4)  # logistic regression model with regularization. Adding regularization term is to reduce over-fitting
    log = model.fit(X_train, y_train)  
    return log.score(X_test, y_test), log.predict(X_test), y_test

In [ ]:
log_score_sppmi, y_pred, y_truth = evaluate_log_model(vecs_sppmi, reviews_times, y)

In [ ]:
log_score_sppmi

In [ ]:
log_score_sgn, y_pred, y_truth = evaluate_log_model(vecs_sgns, reviews_times, y)

In [ ]:
log_score_sgn

In [ ]:
#This model takes a long time to run
def evaluate_boosted_model(X, review_times, y):
    #X = np.hstack((X, review_times))
    X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.2, random_state = 195)

    model = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=100, 
                               subsample=1.0, criterion='friedman_mse', min_samples_split=2,
                               min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, 
                               min_impurity_decrease=0.0, tol=0.01) #train a boosted tree model
    boosted = model.fit(X_train, y_train) 
    return boosted.score(X_test, y_test), boosted.predict(X_test) , y_test

In [ ]:
boosted_score, y_pred_boost, y_truth =  evaluate_boosted_model(vecs_sppmi, reviews_times, y)

In [ ]:
boosted_score  #accuracy score of gradient boosting using vecs_sppmi

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
y_emsem =[] # combine the results from the above two methods. 
for i in range(len(y_truth)):
    y_emsem.append(int(0.8*y_pred[i]+y_pred_boost[i]*0.2))

#The mixture doesn't really work, which makes sense because combining categorical labels doesn't really have a clear meaning. 
#Mixture model might have a better performance when the predicted results are continunous or probability.

In [ ]:
accuracy_score(y_emsem, y_truth) #accuracy score of mixed y label

In [ ]:
# trying dimension reduction results using logistic regression. Doesn't improve the performance
accuracy = []
model =  LsiModel(reviews_bow, id2word = reviews_dict, num_topics = 100)
V = model[reviews_bow]
dense_V = densify(V, 100)
#acc = evaluate_model(dense_V, reviews_times, y)
log_score2, y_pred2, y_truth = evaluate_log_model(dense_V, reviews_times, y)


In [ ]:
log_score2 # accuracy score of dimension reducted feature

In [ ]:
#To conclude, among random forest, neural network, k-means, logistic regression and gradient boosted tree, 
#we see that random forest provides a relatively good baseline, while
# k-means and neural networks seem not to be suitable for this task. 
#It might also due to the neural network structure. 
#Logistic regression and gradient boosting improved the performance a lot. 
#Logistic regression seems to be the best model.